In [1]:
import os
import numpy as np
from skimage import io, measure
from skimage.measure import regionprops
import glob
from ultralytics import YOLO

import torch
import napari
import skimage as ski
import pandas as pd
import plotly.express as px

In [2]:
viewer = napari.Viewer()

In [3]:
current_dir = 'S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130719/'

In [4]:
variant = '36'

In [5]:
model = YOLO('runs/detect/train'+variant+'/weights/best.pt')  # load a custom model

In [6]:
def pad_to_window(img, window=10000):
    height, width = img.shape[0:2]
    new_img = np.pad(img, ((0, window-height), (0, window-width), (0,0)), mode='constant', constant_values=255)
    return new_img
def window_images(img, window=10000):
    shape = img.shape[0:2]
    y_comb = []
    for y in range(0, shape[0], window):
        x_comb = []
        for x in range(0, shape[1], window):
            x_comb.append(pad_to_window(img[y:y+window, x:x+window]))
        y_comb.append(x_comb)
    return np.array(y_comb)

def de_window(img):
    rtn_img = np.concatenate([np.concatenate([x for x in simg], axis=1) for simg in img], axis=0)
    return rtn_img
def process_file(fname, display=False, viewer=None, confidence_cutoff=0.5):
    img = ski.io.imread(fname)
    windowed = window_images(img)
    confs = []
    rtn_boxes = []
    classes = []
    for y,yimg in enumerate(windowed):
        for x,ximg in enumerate(yimg):
            height = 10000
            width = 10000
            results = model(ximg)
            for box in results[0].boxes:
                xyxy = np.reshape(box.xyxy[0].cpu().numpy(), [-1,2])[:,::-1]
                xyxy = xyxy + np.array([y*10000, x*10000])[np.newaxis,:]
                rtn_boxes.append(xyxy)
                classes.append(box.cls.cpu().numpy()[0])
                confs.append(box.conf[0].cpu().numpy())
    classes = np.array(classes)
    rtn_boxes = np.array(rtn_boxes)
    confs = np.array(confs)

    classes = classes[confs > confidence_cutoff]
    rtn_boxes = rtn_boxes[confs > confidence_cutoff]
    confs = confs[confs > confidence_cutoff]
    rtn_img = de_window(windowed)

    if display:
        viewer.layers.clear()
        viewer.add_image(rtn_img, channel_axis=-1, colormap=['red', 'green', 'blue'])
        viewer.add_shapes(rtn_boxes[classes==0], shape_type='rectangle', edge_color='red', face_color='red', opacity=0.5, name='T0')
        viewer.add_shapes(rtn_boxes[classes==1], shape_type='rectangle', edge_color='blue', face_color='blue', opacity=0.5, name='T1')
        viewer.add_shapes(rtn_boxes[classes==2], shape_type='rectangle', edge_color='green', face_color='green', opacity=0.5, name='T2')
        viewer.add_shapes(rtn_boxes[classes==3], shape_type='rectangle', edge_color='yellow', face_color='yellow', opacity=0.5, name='T3')

    return rtn_img, rtn_boxes, classes, confs



In [7]:
df = pd.read_csv(current_dir + 'AllCounts_'+variant+'.csv')
df['class'] = df['class'].astype(str)

In [8]:
df['Tray'] = df['file'].str.split('\\').str[-1].str.split('_').str[0].astype(int)
df['Slide'] = df['file'].str.split('\\').str[-1].str.split('_').str[1].astype(int)
df['Object'] = df['file'].str.split('\\').str[-1].str.split('_').str[2].str[0:2].astype(int)

In [9]:
mapper = pd.read_csv(current_dir+'Mapper.csv')
mapper['Slide'] = mapper['slide'].str.split('Slide').str[-1].astype(int)

In [10]:
df = df.merge(mapper, on='Slide', how='left')
df['Genotype'] = df['Primary Name'].str.split('_').str[0]
df

,Unnamed: 0,file,class,conf,area,Tray,Slide,Object,slide,Primary Name,Secondary Name,Genotype
0,0,S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130...,1.0,0.990160,181803.248614,1,1,2,Slide1,Surface_3.240.3_490dpf,HS_24-243-1-1,Surface
1,1,S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130...,1.0,0.989733,204988.475932,1,1,2,Slide1,Surface_3.240.3_490dpf,HS_24-243-1-1,Surface
2,2,S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130...,1.0,0.986687,195808.801430,1,1,2,Slide1,Surface_3.240.3_490dpf,HS_24-243-1-1,Surface
3,3,S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130...,1.0,0.986123,167154.988437,1,1,2,Slide1,Surface_3.240.3_490dpf,HS_24-243-1-1,Surface
4,4,S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130...,1.0,0.975761,209901.506563,1,1,2,Slide1,Surface_3.240.3_490dpf,HS_24-243-1-1,Surface
...,...,...,...,...,...,...,...,...,...,...,...,...
16551,151,S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130...,0.0,0.524895,33423.645562,1,45,5,Slide45,Molino_5.69.1_497dpf,HS_24-243-3-15,Molino
16552,152,S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130...,3.0,0.524579,64813.206717,1,45,5,Slide45,Molino_5.69.1_497dpf,HS_24-243-3-15,Molino
16553,153,S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130...,0.0,0.523380,23864.862865,1,45,5,Slide45,Molino_5.69.1_497dpf,HS_24-243-3-15,Molino
16554,154,S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130...,0.0,0.519172,17127.124445,1,45,5,Slide45,Molino_5.69.1_497dpf,HS_24-243-3-15,Molino


In [11]:
df.to_csv('S:/micro/nro/fx2482/lem/smc/csv_files/20240820.csv')

In [12]:
agged = df.groupby(['Genotype', 'class']).agg({'conf': 'count', 'area':'median'}).reset_index().rename(columns={'conf': 'Count'})
agged_sum = agged.groupby('Genotype').agg({'Count': 'sum'}).reset_index().rename(columns={'Count': 'Total'})
agged = pd.merge(agged, agged_sum, on='Genotype')
agged['Fraction'] = agged['Count'] / agged['Total']

In [13]:
color_map = {
    '0.0': 'red',
    '1.0': 'blue',
    '3.0': 'yellow',
    # Add more classes and colors as needed
}

category_orders = {'class':['0.0', '1.0', '3.0'], 'Genotype':['Surface', 'Pachon', 'Molino']}


In [14]:
f = px.bar(agged, x='Genotype', y='Fraction', color='class', color_discrete_map=color_map, category_orders=category_orders, width=600)
f.write_html(current_dir + 'Fractions_'+variant+'.html')
f.write_image(current_dir + 'Fractions_'+variant+'.png')
f

In [15]:
f = px.box(df, x='Genotype', y='area', facet_col='class', color='class', color_discrete_map=color_map, category_orders=category_orders, width=800)
f.write_html(current_dir + 'Areas_'+variant+'.html')
f.write_image(current_dir + 'Areas_'+variant+'.png')
f

In [16]:
agged = df.groupby(['file', 'class', 'Genotype']).agg({'conf': 'count'}).reset_index()

In [17]:
import plotly.graph_objects as go

agged['f'] = agged['file'].str.split('\\').str[-1]


f=go.FigureWidget(
    px.bar(agged, x='f', y='conf', color='class', height=800, hover_data=['file'], color_discrete_map=color_map, category_orders=category_orders, width=1600, facet_col='Genotype')
    )

def click_fn(trace, points, state):
    
    if (len(points.point_inds)>0):
        idx = f.data[points.trace_index]['customdata'][points.point_inds[-1]][0]
        print(idx)
        process_file(idx, display=True, viewer=viewer, confidence_cutoff=0.05)

for a in f.data:
    a.on_click(click_fn)
f.write_html(current_dir + 'Counts_'+variant+'.html')
f.write_image(current_dir + 'Counts_'+variant+'.png')
f

FigureWidget({
    'data': [{'alignmentgroup': 'True',
              'customdata': array([['S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130719\\01_01_02.tif'],
                                   ['S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130719\\01_01_03.tif'],
                                   ['S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130719\\01_02_02.tif'],
                                   ['S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130719\\01_02_03.tif'],
                                   ['S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130719\\01_03_02.tif'],
                                   ['S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130719\\01_03_03.tif'],
                                   ['S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130719\\01_04_02.tif'],
                                   ['S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130719\\01_04_03.tif'],
                                   ['S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130719\\01_05_02.tif'],
         

S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130719\01_22_02.tif

0: 1600x1600 95 1_stage_IBs, 4 2_stage_IIs, 63 4_stage_late_IIIs, 32.1ms
Speed: 27.6ms preprocess, 32.1ms inference, 2.0ms postprocess per image at shape (1, 3, 1600, 1600)
S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130719\01_07_03.tif

0: 1600x1600 93 1_stage_IBs, 18 2_stage_IIs, 49 4_stage_late_IIIs, 305.4ms
Speed: 36.4ms preprocess, 305.4ms inference, 10.3ms postprocess per image at shape (1, 3, 1600, 1600)


In [17]:
df.shape

(16556, 12)